In [1]:
from recompilation_functions import *
from randomized_compiling_functions import *
from ZNE import *

C:\Users\hugop\AppData\Roaming\Python\Python39\site-packages\numba\np\ufunc\parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12040. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
import os
nr=100#number of disorder realizations
t=10#time

for i in range(1,nr+1):
    if not os.path.exists('data/circuits/disorder_realization_'+str(i)+'/T='+str(round(t,1))+'/ZNE3/'):
        os.makedirs('data/circuits/disorder_realization_'+str(i)+'/T='+str(round(t,1))+'/ZNE3/')


In [ ]:
#creation of the circuits using recompiled tensor network method

nb_qubits = 4#system size
depth=20 #depth of the recompiled circuit
t = 10#time of the simulation (quench)
mean=0 #mean value of the coupling strength's gaussian distribution
sigma=1 #standard deviation of the coupling strength's gaussian distribution
nr=100 #number of disorder realizations
omegas=[0.28,0.38,0.63,0.86] #qubits frequencies


r=1#initial disorder realization
for d in range(r,nr+1):


    #random interaction
    J=np.zeros((nb_qubits,nb_qubits),dtype=complex)
    for i in range(nb_qubits):
        for j in range(i+1,nb_qubits):
            J[i,j]= np.random.normal(loc = mean, scale = np.sqrt(0.5 * sigma**2/nb_qubits), size = (1, 2)).view(np.complex)[0][0]
            J[j,i]=np.conjugate(J[i,j])

    #Saving the matrix of random coupling_strength of the system for one disorder realization
    #WARNING IF YOU UNCOMMENT THE LINE BELOW, THE PREVIOUS MATRIX OF THE CORRESPONDING DISORDER REALIZATION WILL BE ERASED
    #np.save('data/disorder_realization_'+str(d)+'/random_interaction_'+str(d),J)

    np.set_printoptions(precision=5)
    print(d)
    print(J)

    #exact unitary
    unitary = linalg.expm(-1j*random_gaussian_interactions(J,omegas)*t)

    
    #initial state
    qc=QuantumCircuit(nb_qubits)
    for i in range(nb_qubits):
        qc.rx(np.pi/2,i)
    qc.barrier()
    
    #circuit recompiled
    qc_recompiled=tensor_network_recompilation(unitary, depth, ansatz_fct = ansatz_circuit, two_qubit_gate = 'CX', n_iter = 5000, n_hop = 20, temperature = 10, print_bool = True)  
    qc.append(qc_recompiled,[i for i in range(nb_qubits)])
    
    #measurement in the y-direction
    qc_meas=add_measure(qc)

    #save the quantum circuit
    #WARNING IF YOU UNCOMMENT THE LINE BELOW, THE PREVIOUS CIRCUIT OF THE CORRESPONDING DISORDER REALIZATION WILL BE ERASED
    #qc_meas.qasm(formatted=True,filename='data/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/recompiled_circuit_disorder='+str(d)+'_T='+str(round(t,1))+'.qasm')

In [3]:
#creation of the circuits using recompiled tensor network method

nb_qubits = 4#system size
depth=20 #depth of the recompiled circuit
t = 10#time of the simulation (quench)
mean=0 #mean value of the coupling strength's gaussian distribution
sigma=1 #standard deviation of the coupling strength's gaussian distribution
nr=100 #number of disorder realizations
omegas=[0.28,0.38,0.63,0.86] #qubits frequencies

spin_up=[0,1,2,3]
initial_state='highest_energy_state'

r=1#initial disorder realization
for d in range(r,nr+1):


    #random interaction
    
    #load the matrix of random coupling_strength
    J=np.load('data/circuits/disorder_realization_'+str(d)+'/random_interaction_'+str(d)+'.npy')

    np.set_printoptions(precision=5)
    print(d)
    print(J)

    #exact unitary
    unitary = linalg.expm(-1j*random_gaussian_interactions(J,omegas)*t)

    
    #initial state
    
    qc=QuantumCircuit(nb_qubits)
    for i in range(nb_qubits):
        if i in spin_up:
            qc.rx(-np.pi/2,i)
        else:
            qc.rx(np.pi/2,i)
    qc.barrier()
    
    #circuit recompiled
    qc_recompiled=tensor_network_recompilation(unitary, depth, ansatz_fct = ansatz_circuit, two_qubit_gate = 'CX', n_iter = 5000, n_hop = 20, temperature = 10, print_bool = True)  
    qc.append(qc_recompiled,[i for i in range(nb_qubits)])
    
    #measurement in the y-direction
    qc_meas=add_measure(qc)

    #save the quantum circuit
    #WARNING IF YOU UNCOMMENT THE LINE BELOW, THE PREVIOUS CIRCUIT OF THE CORRESPONDING DISORDER REALIZATION WILL BE ERASED
    #qc_meas.qasm(formatted=True,filename='data/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/recompiled_circuit_'+initial_state+'_disorder_'+str(d)+'_T='+str(round(t,1))+'.qasm')

1
[[ 0.00000e+00+0.j       5.49416e-03+0.00284j  2.04756e-01+0.32497j
  -6.67014e-02-0.3167j ]
 [ 5.49416e-03-0.00284j  0.00000e+00+0.j      -3.47428e-04-0.08453j
  -6.96387e-01-0.47774j]
 [ 2.04756e-01-0.32497j -3.47428e-04+0.08453j  0.00000e+00+0.j
   6.56541e-02+0.4465j ]
 [-6.67014e-02+0.3167j  -6.96387e-01+0.47774j  6.56541e-02-0.4465j
   0.00000e+00+0.j     ]]
Original number of CNOT : 241
Ansatz number of CNOT : 60


+0.006156444550 [best: +0.003074645996] :   9%|██▉                              | 8971/100000 [01:12<12:20, 122.95it/s]
C:\Users\hugop\AppData\Roaming\Python\Python39\site-packages\autoray\autoray.py:1052: UserWarning: Explicitly requested dtype <class 'jax.numpy.complex128'> requested in astype is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(dtype, **kwargs)


Fidelity (tensor): 99.7112156961 %
Fidelity (circuit): 99.7108039475 %
OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg meas[4];
rz(pi/2) q[0];
sx q[0];
rz(3*pi/2) q[0];
sx q[0];
rz(5*pi/2) q[0];
rz(pi/2) q[1];
sx q[1];
rz(3*pi/2) q[1];
sx q[1];
rz(5*pi/2) q[1];
rz(pi/2) q[2];
sx q[2];
rz(pi/2) q[2];
sx q[2];
rz(5*pi/2) q[2];
rz(pi/2) q[3];
sx q[3];
rz(pi/2) q[3];
sx q[3];
rz(5*pi/2) q[3];
barrier q[0],q[1],q[2],q[3];
rz(0.28119170665741) q[0];
sx q[0];
rz(3.59086138208444) q[0];
sx q[0];
rz(9.75912970899745) q[0];
rz(0.457625806331635) q[1];
sx q[1];
rz(3.96982249815995) q[1];
sx q[1];
rz(10.4258756399076) q[1];
cx q[1],q[0];
rz(-0.178925737738609) q[0];
sx q[0];
rz(4.14151057799394) q[0];
sx q[0];
rz(9.41460673174962) q[0];
rz(0.260920524597168) q[2];
sx q[2];
rz(3.7642818411165) q[2];
sx q[2];
rz(10.9796041011731) q[2];
cx q[2],q[1];
rz(-0.567291736602783) q[1];
sx q[1];
rz(2.69174471695954) q[1];
sx q[1];
rz(9.56730111538574) q[1];
rz(1.90598380565643) q[3];
sx q[3];
rz(2.901331

Original number of CNOT : 268
Ansatz number of CNOT : 60


+0.003308117390 [best: +0.002977728844] :   9%|███                              | 9386/100000 [01:14<11:54, 126.76it/s]


...

Original number of CNOT : 271
Ansatz number of CNOT : 60


+0.005153834820 [best: +0.002788901329] :  10%|███▎                            | 10236/100000 [01:19<11:37, 128.67it/s]


Fidelity (tensor): 99.7375198108 %
Fidelity (circuit): 99.7383419669 %
OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg meas[4];
rz(pi/2) q[0];
sx q[0];
rz(3*pi/2) q[0];
sx q[0];
rz(5*pi/2) q[0];
rz(pi/2) q[1];
sx q[1];
rz(3*pi/2) q[1];
sx q[1];
rz(5*pi/2) q[1];
rz(pi/2) q[2];
sx q[2];
rz(pi/2) q[2];
sx q[2];
rz(5*pi/2) q[2];
rz(pi/2) q[3];
sx q[3];
rz(pi/2) q[3];
sx q[3];
rz(5*pi/2) q[3];
barrier q[0],q[1],q[2],q[3];
rz(0.754741549491882) q[0];
sx q[0];
rz(4.19552245934541) q[0];
sx q[0];
rz(10.0007458090703) q[0];
rz(4.24813461303711) q[1];
sx q[1];
rz(2.6701789518171) q[1];
sx q[1];
rz(9.85195863842174) q[1];
cx q[1],q[0];
rz(3.70858287811279) q[0];
sx q[0];
rz(2.40752527316148) q[0];
sx q[0];
rz(12.6993951558988) q[0];
rz(0.686049342155457) q[2];
sx q[2];
rz(1.71608701546723) q[2];
sx q[2];
rz(8.67670348881885) q[2];
cx q[2],q[1];
rz(1.88153028488159) q[1];
sx q[1];
rz(4.04290989239747) q[1];
sx q[1];
rz(8.35919044017001) q[1];
rz(0.159473314881325) q[3];
sx q[3];
rz(6.261205108

Original number of CNOT : 280
Ansatz number of CNOT : 60


+0.004871308804 [best: +0.002424180508] :   3%|▉                                | 2926/100000 [00:35<09:12, 175.77it/s]

In [2]:
%%capture
#creation of the ZNE circuits with tripled CNOT gates for each disorder realization

spin_up=[0,1,2,3]
initial_state='highest_energy_state'

nr=100#number of disorder realizations
r=1
t=10 #time of the simulation 
nb_twirls=100#number of RC circuits per realization
coupling_map=[[0,1],[1,0],[1,2],[2,1],[2,3],[3,2]]#linear coupling map between the qubits
for d in range(r,nr+1):
    qc=QuantumCircuit.from_qasm_file('data/circuits/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/recompiled_circuit_'+initial_state+'_disorder_'+str(d)+'_T='+str(round(t,1))+'.qasm')
    for k in range(nb_twirls):
        qc2=crosstalk_twirl(qc,coupling_map)
        #save circuits, if you uncomment the line below the previous circuits already saved will be erased
        #qc2.qasm(formatted=True,filename='data/ circuits/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/RC/recompiled_circuit_'+initial_state+'_disorder_'+str(d)+'_T='+str(round(t,1))+'_RC_'+str(k)+'.qasm')

In [2]:
%%capture
nr=100
r=1
t=10
initial_state='highest_energy_state'
nb_twirls=100
coupling_map=[[0,1],[1,0],[1,2],[2,1],[2,3],[3,2]]
for d in range(r,nr+1):
    qc=QuantumCircuit.from_qasm_file('data/circuits/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/recompiled_circuit_'+initial_state+'_disorder_'+str(d)+'_T='+str(round(t,1))+'.qasm')
    for k in range(nb_twirls):
        qc2=bruit(qc,3)
        qc3=crosstalk_twirl(qc2,coupling_map)
        #save circuits, if you uncomment the line below the previous circuits already saved will be erased
        
        qc3.qasm(formatted=True,filename='data/circuits/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/ZNE3/recompiled_circuit_'+initial_state+'_disorder_'+str(d)+'_T='+str(round(t,1))+'_ZNE3_'+str(k)+'.qasm')